# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [16]:

df_model = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])

df_model = df_model.dropna()

formula = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + ' \
          'educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

y, X = patsy.dmatrices(formula, data=df_model, return_type='dataframe')

modelo = sm.OLS(y, X).fit()

print(modelo.summary())


                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 02 May 2025   Prob (F-statistic):               0.00
Time:                        10:52:41   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [22]:
formula_reduzida = """
np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel +
                C(tipo_renda, Treatment(reference="Assalariado")) +
                C(estado_civil, Treatment(reference="Casado")) +
                qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia
"""

df_model = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])
df_model = df_model.dropna()

y_reduzido, X_reduzido = patsy.dmatrices(formula_reduzida, data=df_model, return_type='dataframe')

formula_reduzida = sm.OLS(y_reduzido, X_reduzido).fit()

print(formula_reduzida.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     523.9
Date:                Fri, 02 May 2025   Prob (F-statistic):               0.00
Time:                        11:01:56   Log-Likelihood:                -16262.
No. Observations:               15000   AIC:                         3.256e+04
Df Residuals:                   14984   BIC:                         3.268e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

O primeiro modelo apresentou um melhor AIC e R2, o que leva a crer que antes da retirada das variáveis não significativas tinha uma melhor qualidade de previsão.

A maior alteração foi no tipo de renda, onde teve valores com mais discrepância apos a alteração.

In [34]:
formula_reduzida = """
np.log(renda) ~ sexo + posse_de_veiculo +
                C(tipo_renda, Treatment(reference="Assalariado")) +
                C(estado_civil, Treatment(reference="Casado")) +
                qtd_filhos + tempo_emprego + qt_pessoas_residencia
"""

df_model = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])
df_model = df_model.dropna()

y_reduzido, X_reduzido = patsy.dmatrices(formula_reduzida, data=df_model, return_type='dataframe')

formula_reduzida = sm.OLS(y_reduzido, X_reduzido).fit()

print(formula_reduzida.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     592.9
Date:                Fri, 02 May 2025   Prob (F-statistic):               0.00
Time:                        11:13:43   Log-Likelihood:                -16312.
No. Observations:               15000   AIC:                         3.265e+04
Df Residuals:                   14986   BIC:                         3.276e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

O melhor modelo que esta apresentando continua sendo o primeiro com mais variaveis.